# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import pandas as pd 
import numpy as np

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
import os
os.chdir('/content/drive/My Drive/FNC/')

In [6]:
!ls

glove.6B.100d.txt  glove.6B.zip		       train_bodies.gsheet
glove.6B.200d.txt  test_bodies.csv	       train_stances.csv
glove.6B.300d.txt  test_stances_unlabeled.csv
glove.6B.50d.txt   train_bodies.csv


In [0]:
project_path = '/content/drive/My Drive/FNC/'

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
# Importing 'train_bodies'
df_train_bodies = pd.read_csv(project_path+'train_bodies.csv')

In [10]:
# Getting a glance of above data
df_train_bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [11]:
df_train_bodies.shape

(1683, 2)

In [0]:
# Importing 'train_stances'
df_train_stances = pd.read_csv(project_path+'train_stances.csv')

In [13]:
# Getting a glance of above data
df_train_stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [14]:
df_train_stances.shape

(49972, 3)

In [0]:
#Merging both the dataset basis variable 'Body ID'

dataset = pd.merge(df_train_bodies, df_train_stances)


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [16]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [19]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [0]:
dataset['articleBody'] = dataset['articleBody'].apply(lambda x: x.lower())
dataset['articleBody'] = dataset['articleBody'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
tokenizer.fit_on_texts(dataset['articleBody'].values)
X1 = tokenizer.texts_to_sequences(dataset['articleBody'].values)
X1 = pad_sequences(X1)

In [0]:
X1

In [0]:
dataset['Headline'] = dataset['Headline'].apply(lambda x: x.lower())
dataset['Headline'] = dataset['Headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
tokenizer.fit_on_texts(dataset['Headline'].values)
X2 = tokenizer.texts_to_sequences(dataset['Headline'].values)
X2 = pad_sequences(X2)

In [0]:
X2

In [25]:
# print(tokenizer.word_counts)
# print(tokenizer.word_docs)
print(tokenizer.word_index)
# print(tokenizer.document_count)
# encoded_text = tokenizer.texts_to_matrix(text)
# print(encoded_text)

{'the': 1, 'to': 2, 'a': 3, 'of': 4, 'in': 5, 'and': 6, 'that': 7, 'is': 8, 'was': 9, 'on': 10, 'for': 11, 'said': 12, 'he': 13, 'it': 14, 'with': 15, 'his': 16, 'have': 17, 'as': 18, 'by': 19, 'has': 20, 'from': 21, 'at': 22, 'be': 23, 'an': 24, 'not': 25, 'are': 26, 'been': 27, 'but': 28, 'this': 29, 'who': 30, 'had': 31, 'they': 32, 'i': 33, 'its': 34, 'we': 35, 'were': 36, 'after': 37, 'will': 38, 'about': 39, 'one': 40, 'us': 41, 'or': 42, 'which': 43, 'she': 44, 'video': 45, 'isis': 46, 'apple': 47, 'her': 48, 'would': 49, 'when': 50, 'their': 51, 'state': 52, 'also': 53, 'more': 54, 'up': 55, 'told': 56, 'no': 57, 'new': 58, 'all': 59, 'out': 60, 'people': 61, 'there': 62, 'you': 63, 'if': 64, 'him': 65, 'news': 66, 'what': 67, 'islamic': 68, 'man': 69, 'could': 70, 'some': 71, 'watch': 72, 'over': 73, 'time': 74, 'according': 75, 'group': 76, 'into': 77, 'so': 78, 'just': 79, 'being': 80, 'last': 81, 'reports': 82, 'first': 83, 'police': 84, 'syria': 85, 'them': 86, 'government

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [32]:
# texts = []

# articles = []

# sent_tokenize()


texts = dataset['Headline'].tolist()


import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
joint_words = ' '.join(texts)

articles = word_tokenize(joint_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Check 2:

first element of texts and articles should be as given below. 

In [30]:
texts[0]

'soldier shot parliament locked down after gunfire erupts at war memorial'

In [35]:
articles[0]

'soldier'

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
tokenizer.texts_to_sequences(dataset['Headline'].values)

### Check 3:

Accessing first element in data should give something like given below.

In [0]:
data[0, :, :]

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [0]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [0]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

### Model

### Compile and fit the model

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)